# kernel : ml-dl-nlp
# 파일 read

In [82]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
data = pd.read_pickle('data/industry_labeled.pickle')
stock_data = pd.read_csv('data/주가지수크롤링(증감치숫자로).csv')
data.sort_values(by=['산업','날짜'],ascending=[True,True], inplace=True)
data["날짜"] = pd.to_datetime(data["날짜"]).dt.date

build_data = data[data['산업'].values=='건설']
car_data = data[data['산업'].values=='자동차']
health_data = data[data['산업'].values=='헬스케어']

stock_data['날짜']= pd.to_datetime(stock_data['날짜'].astype(str), format="%Y%m%d")
stock_data.info()
# build_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  174 non-null    int64         
 1   날짜          174 non-null    datetime64[ns]
 2   종목명         174 non-null    object        
 3   종가(백만원)     174 non-null    float64       
 4   거래량(천주)     174 non-null    int64         
 5   전일대비        174 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 8.3+ KB


# 감정분석 모델

In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 감정분석 모델 로딩
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBERT")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBERT")

def get_sentiment_probs(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=-1).detach().numpy()[0]
    return probs  # [p_neg, p_pos]

# 건설 감정분석

In [96]:
import pandas as pd
import numpy as np
from datetime import timedelta

def build_sentiment(build_data, stock_data, start_date, end_date):
    # 날짜 범위 계산
    diff_date = (end_date - start_date).days
    check_date = start_date

    # 거래일 리스트 준비
    stock_data['날짜'] = pd.to_datetime(stock_data['날짜']).dt.date
    trading_days = sorted(stock_data['날짜'].unique())

    def map_to_next_trading_day(d):
        # d 이후의 거래일 중 가장 가까운 날 찾기
        future_days = [day for day in trading_days if day > d]
        if len(future_days) == 0:
            return None
        return min(future_days)

    # 최종 결과 DataFrame
    final_df = pd.DataFrame(columns=['거래일', '종목명', 'sentiment', '전일대비'])

    for i in range(diff_date):
        # 건설 뉴스만 추출
        b_news_df = build_data[build_data['날짜'] == check_date]
        if b_news_df.empty:
            check_date += timedelta(days=1)
            continue

        cols = ['날짜', '제목', '본문', '링크', '산업']
        news_df = b_news_df[cols].copy()

        # 뉴스 감정 분석
        news_df['sentiment'] = news_df['제목'].apply(get_sentiment_probs)

        # 하루치 기사별 평균 감정 확률
        daily_sentiment = news_df.groupby('날짜')['sentiment'].apply(
            lambda x: np.mean(np.stack(x), axis=0)
        ).reset_index()

        # 타입 맞추기
        daily_sentiment['날짜'] = pd.to_datetime(daily_sentiment['날짜']).dt.date

        # 주말 뉴스 → 다음 거래일 매핑
        daily_sentiment['거래일'] = daily_sentiment['날짜'].apply(map_to_next_trading_day)

        # 거래일 없는 경우 제외
        daily_sentiment = daily_sentiment.dropna(subset=['거래일'])
        if daily_sentiment.empty:
            check_date += timedelta(days=1)
            continue

        # 건설 종목만 병합
        b_stock_df = stock_data[stock_data['종목명'] == 'KRX 건설']
        daily_sentiment = pd.merge(
            daily_sentiment,
            b_stock_df,
            left_on=daily_sentiment['거래일'].apply(pd.Timestamp), 
            right_on=pd.to_datetime(b_stock_df['날짜']),
            how='left'
        )

        # 원하는 열만 정리
        cols = ['거래일', '종목명', 'sentiment', '전일대비']
        daily_sentiment = daily_sentiment[cols]

        # 결과 누적
        final_df = pd.concat([final_df, daily_sentiment], ignore_index=True)

        check_date += timedelta(days=1)

    # 날짜 기준으로 정렬
    if not final_df.empty:
        final_df = final_df.sort_values(by='거래일').reset_index(drop=True)

        # 이전 행과 전일대비 값 비교 → 중복 제거
        mask = final_df['전일대비'] != final_df['전일대비'].shift(1)
        final_df = final_df[mask]

        return final_df
    else:
        print("결과가 없습니다.")
        return pd.DataFrame()


In [97]:
start_date = pd.to_datetime("2025-12-01")
end_date = pd.to_datetime("2025-12-21")

build_final_df = build_sentiment(build_data, stock_data, start_date, end_date)
print(build_final_df)

           거래일     종목명                 sentiment   전일대비
0   2025-12-02  KRX 건설   [0.6357966, 0.36420333]   9.28
1   2025-12-03  KRX 건설   [0.6399582, 0.36004186]  24.01
2   2025-12-04  KRX 건설   [0.6402304, 0.35976946]  -9.54
3   2025-12-05  KRX 건설  [0.63413376, 0.36586633]  36.51
4   2025-12-08  KRX 건설   [0.63741994, 0.3625799] -13.97
7   2025-12-09  KRX 건설   [0.6423001, 0.35769966]  -7.25
8   2025-12-10  KRX 건설    [0.633504, 0.36649597]  -2.82
9   2025-12-11  KRX 건설   [0.6367126, 0.36328742]  18.91
10  2025-12-12  KRX 건설   [0.6317937, 0.36820653]  38.62
11  2025-12-15  KRX 건설   [0.6218646, 0.37813544] -30.13
14  2025-12-16  KRX 건설     [0.6379209, 0.362079] -22.74
15  2025-12-17  KRX 건설  [0.64143336, 0.35856652]  13.58
16  2025-12-18  KRX 건설   [0.6311379, 0.36886203] -11.06
17  2025-12-19  KRX 건설   [0.6390875, 0.36091238]   1.14
18  2025-12-22  KRX 건설  [0.63662213, 0.36337787]  17.29


# 자동차 감정분석

In [98]:
def car_sentiment(car_data, stock_data, start_date, end_date):
    # 날짜 범위 계산
    diff_date = (end_date - start_date).days
    check_date = start_date

    # 거래일 리스트 준비
    stock_data['날짜'] = pd.to_datetime(stock_data['날짜']).dt.date
    trading_days = sorted(stock_data['날짜'].unique())

    def map_to_next_trading_day(d):
        # d 이후의 거래일 중 가장 가까운 날 찾기
        future_days = [day for day in trading_days if day > d]
        if len(future_days) == 0:
            return None
        return min(future_days)

    # 최종 결과 DataFrame
    final_df = pd.DataFrame(columns=['거래일', '종목명', 'sentiment', '전일대비'])

    for i in range(diff_date):
        # 자동차 뉴스만 추출
        c_news_df = car_data[car_data['날짜'] == check_date]
        if c_news_df.empty:
            check_date += timedelta(days=1)
            continue

        cols = ['날짜', '제목', '본문', '링크', '산업']
        news_df = c_news_df[cols].copy()

        # 뉴스 감정 분석
        news_df['sentiment'] = news_df['제목'].apply(get_sentiment_probs)

        # 하루치 기사별 평균 감정 확률
        daily_sentiment = news_df.groupby('날짜')['sentiment'].apply(
            lambda x: np.mean(np.stack(x), axis=0)
        ).reset_index()

        # 타입 맞추기
        daily_sentiment['날짜'] = pd.to_datetime(daily_sentiment['날짜']).dt.date

        # 주말 뉴스 → 다음 거래일 매핑
        daily_sentiment['거래일'] = daily_sentiment['날짜'].apply(map_to_next_trading_day)

        # 거래일 없는 경우 제외
        daily_sentiment = daily_sentiment.dropna(subset=['거래일'])
        if daily_sentiment.empty:
            check_date += timedelta(days=1)
            continue

        # 자동차 종목만 병합
        c_stock_df = stock_data[stock_data['종목명'] == 'KRX 자동차']
        daily_sentiment = pd.merge(
            daily_sentiment,
            c_stock_df,
            left_on=daily_sentiment['거래일'].apply(pd.Timestamp), 
            right_on=pd.to_datetime(b_stock_df['날짜']),
            how='left'
        )

        # 원하는 열만 정리
        cols = ['거래일', '종목명', 'sentiment', '전일대비']
        daily_sentiment = daily_sentiment[cols]

        # 결과 누적
        final_df = pd.concat([final_df, daily_sentiment], ignore_index=True)

        check_date += timedelta(days=1)

    # 날짜 기준으로 정렬
    if not final_df.empty:
        final_df = final_df.sort_values(by='거래일').reset_index(drop=True)

        # 이전 행과 전일대비 값 비교 → 중복 제거
        mask = final_df['전일대비'] != final_df['전일대비'].shift(1)
        final_df = final_df[mask]

        return final_df
    else:
        print("결과가 없습니다.")
        return pd.DataFrame()


In [99]:
start_date = pd.to_datetime("2025-12-01")
end_date = pd.to_datetime("2025-12-21")

car_final_df = car_sentiment(build_data, stock_data, start_date, end_date)
print(car_final_df)

           거래일      종목명                 sentiment    전일대비
0   2025-12-02  KRX 자동차   [0.6357966, 0.36420333]   81.63
1   2025-12-03  KRX 자동차   [0.6399582, 0.36004186]   39.31
2   2025-12-04  KRX 자동차   [0.6402304, 0.35976946]   71.24
3   2025-12-05  KRX 자동차  [0.63413376, 0.36586633]  120.77
4   2025-12-08  KRX 자동차   [0.63741994, 0.3625799]   -2.78
7   2025-12-09  KRX 자동차   [0.6423001, 0.35769966]  -45.02
8   2025-12-10  KRX 자동차    [0.633504, 0.36649597]  -12.24
9   2025-12-11  KRX 자동차   [0.6367126, 0.36328742]  -30.82
10  2025-12-12  KRX 자동차   [0.6317937, 0.36820653]   62.99
11  2025-12-15  KRX 자동차   [0.6218646, 0.37813544]  -46.82
14  2025-12-16  KRX 자동차     [0.6379209, 0.362079]  -47.17
15  2025-12-17  KRX 자동차  [0.64143336, 0.35856652]   23.76
16  2025-12-18  KRX 자동차   [0.6311379, 0.36886203]  -50.34
17  2025-12-19  KRX 자동차   [0.6390875, 0.36091238]   56.76
18  2025-12-22  KRX 자동차  [0.63662213, 0.36337787]    7.47


# 헬스케어 감정분석

In [100]:
def health_sentiment(car_data, stock_data, start_date, end_date):
    # 날짜 범위 계산
    diff_date = (end_date - start_date).days
    check_date = start_date

    # 거래일 리스트 준비
    stock_data['날짜'] = pd.to_datetime(stock_data['날짜']).dt.date
    trading_days = sorted(stock_data['날짜'].unique())

    def map_to_next_trading_day(d):
        # d 이후의 거래일 중 가장 가까운 날 찾기
        future_days = [day for day in trading_days if day > d]
        if len(future_days) == 0:
            return None
        return min(future_days)

    # 최종 결과 DataFrame
    final_df = pd.DataFrame(columns=['거래일', '종목명', 'sentiment', '전일대비'])

    for i in range(diff_date):
        # 건설 뉴스만 추출
        h_news_df = health_data[health_data['날짜'] == check_date]
        if h_news_df.empty:
            check_date += timedelta(days=1)
            continue

        cols = ['날짜', '제목', '본문', '링크', '산업']
        news_df = h_news_df[cols].copy()

        # 뉴스 감정 분석
        news_df['sentiment'] = news_df['제목'].apply(get_sentiment_probs)

        # 하루치 기사별 평균 감정 확률
        daily_sentiment = news_df.groupby('날짜')['sentiment'].apply(
            lambda x: np.mean(np.stack(x), axis=0)
        ).reset_index()

        # 타입 맞추기
        daily_sentiment['날짜'] = pd.to_datetime(daily_sentiment['날짜']).dt.date

        # 주말 뉴스 → 다음 거래일 매핑
        daily_sentiment['거래일'] = daily_sentiment['날짜'].apply(map_to_next_trading_day)

        # 거래일 없는 경우 제외
        daily_sentiment = daily_sentiment.dropna(subset=['거래일'])
        if daily_sentiment.empty:
            check_date += timedelta(days=1)
            continue

        # 헬스케어 종목만 병합
        h_stock_df = stock_data[stock_data['종목명'] == 'KRX 헬스케어']
        daily_sentiment = pd.merge(
            daily_sentiment,
            h_stock_df,
            left_on=daily_sentiment['거래일'].apply(pd.Timestamp), 
            right_on=pd.to_datetime(b_stock_df['날짜']),
            how='left'
        )

        # 원하는 열만 정리
        cols = ['거래일', '종목명', 'sentiment', '전일대비']
        daily_sentiment = daily_sentiment[cols]

        # 결과 누적
        final_df = pd.concat([final_df, daily_sentiment], ignore_index=True)

        check_date += timedelta(days=1)

    # 날짜 기준으로 정렬
    if not final_df.empty:
        final_df = final_df.sort_values(by='거래일').reset_index(drop=True)

        # 이전 행과 전일대비 값 비교 → 중복 제거
        mask = final_df['전일대비'] != final_df['전일대비'].shift(1)
        final_df = final_df[mask]

        return final_df
    else:
        print("결과가 없습니다.")
        return pd.DataFrame()


In [101]:
start_date = pd.to_datetime("2025-12-01")
end_date = pd.to_datetime("2025-12-21")

health_final_df = health_sentiment(health_data, stock_data, start_date, end_date)
print(health_final_df)

           거래일       종목명                 sentiment    전일대비
0   2025-12-02  KRX 헬스케어   [0.55993307, 0.4400669]  -37.20
1   2025-12-03  KRX 헬스케어    [0.552401, 0.44759908]   -1.02
2   2025-12-04  KRX 헬스케어   [0.5539471, 0.44605303]   12.03
3   2025-12-05  KRX 헬스케어   [0.5523784, 0.44762158] -130.37
4   2025-12-08  KRX 헬스케어    [0.5563611, 0.4436388]  -54.21
7   2025-12-09  KRX 헬스케어    [0.55866385, 0.441336]   51.86
8   2025-12-10  KRX 헬스케어  [0.55929005, 0.44070995]   41.44
9   2025-12-11  KRX 헬스케어   [0.5605811, 0.43941885]   13.63
10  2025-12-12  KRX 헬스케어    [0.559204, 0.44079605]  -48.01
11  2025-12-15  KRX 헬스케어   [0.55662954, 0.4433706]   62.48
14  2025-12-16  KRX 헬스케어   [0.5564766, 0.44352344]  -46.23
15  2025-12-17  KRX 헬스케어   [0.5568166, 0.44318357]  -85.00
16  2025-12-18  KRX 헬스케어   [0.5624818, 0.43751845]  -20.52
17  2025-12-19  KRX 헬스케어    [0.55295706, 0.447043]   85.57
18  2025-12-22  KRX 헬스케어    [0.556615, 0.44338492]  -44.41


# 여기서 부터는 작업중_BiLSTM딥러닝(날릴수도?)

In [105]:
import torch.nn as nn
import numpy as np

# sentiment를 2개 컬럼으로 분리
build_final_df[['sent_neg','sent_pos']] = pd.DataFrame(
                                    build_final_df['sentiment'].tolist(), index=build_final_df.index)

# 입력 특징 X (sentiment만 사용하거나 sentiment+전일대비 같이 사용 가능)
X = build_final_df[['sent_neg','sent_pos']].values
# 타겟 y (전일대비 → 상승/하락 분류라면 0/1로 변환)
y = (build_final_df['전일대비'] > 0).astype(int).values   # 상승=1, 하락=0


class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim*2, output_dim)  # 양방향이므로 hidden_dim*2

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝 출력
        return out


In [114]:
import torch

# 모델 정의
model = BiLSTM(input_dim=X.shape[1], hidden_dim=32, output_dim=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

losses = []

# 학습 루프
epochs = 100
for epoch in range(epochs):
    outputs = model(X_tensor)          # forward
    loss = criterion(outputs, y_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item()) 
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 10/100, Loss: 0.6930
Epoch 20/100, Loss: 0.6910
Epoch 30/100, Loss: 0.6910
Epoch 40/100, Loss: 0.6910
Epoch 50/100, Loss: 0.6909
Epoch 60/100, Loss: 0.6909
Epoch 70/100, Loss: 0.6909
Epoch 80/100, Loss: 0.6909
Epoch 90/100, Loss: 0.6909
Epoch 100/100, Loss: 0.6909
